In [31]:
# from googletrans import Translator, constants
# from pprint import pprint

# # init the Google API translator
# translator = Translator()
# # translate a spanish text to english text (by default)
# translation = translator.translate("Hola Mundo")
# print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

In [32]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [33]:
def make_soup(url):
    
    request = requests.get(url)
    if request.status_code == 200:
        soup = BeautifulSoup(request.content, 'html.parser')
    else:
        print('failed: ' + request.status_code)
    
    return soup
    
# url = 'https://www.nuerburgring-langstrecken-serie.de/en/calendar-nurburgring-langstrecken-serie-2023/'

In [34]:
soup = make_soup('https://www.nuerburgring-langstrecken-serie.de/en/calendar-nurburgring-langstrecken-serie-2023/')

In [35]:
# temp_tits = []
# count = 1
# for i in range(len(soup.find_all('tr')[1:9])):
#         temp_tits.append('VLN Race R0' + str(count))
#         count += 1
# temp_tits

In [36]:
# soup.find_all('tr')[0]

In [37]:
def scrape_nls(soup):
    
    cln_days = []
    cln_mons = []
    cln_yrs = []
    titles = []
    
    tr = soup.find_all('tr')
    

## For each round item (tr) find date, trim day to race date only (ie 11-13 becomes just 13, fri-sat becomes just sat)
## store dates in clean_dates in format for iCalendar req's
    for i in range(len(soup.find_all('tr'))): 
        temp_date = soup.find_all('tr')[i].find_all('td')[0].text.strip().split('-')[1].split('.')

        temp_day = temp_date[0]
        if temp_day[0] == '0':
            temp_day = temp_day[1:]
        cln_days.append(temp_day)

        temp_mon = temp_date[1]
        if temp_mon[0] == '0':
            temp_mon = temp_mon[1:]
        cln_mons.append(temp_mon)

        cln_yr = temp_date[2]
        cln_yrs.append(cln_yr)

#     clean_dates = []
#     for yr, mon, day in zip(cln_yrs, cln_mons, cln_days):
#         temp_date_ = yr + ', ' + mon + ', ' + day
#         clean_dates.append(temp_date_)
##


## Pull the event names as description and store to description list
    descriptions = []
    for i in range(len(soup.find_all('tr'))):
        temp_desc = soup.find_all('tr')[i].find_all('td')[1].text.strip()
        descriptions.append(temp_desc)
##

## Make a simple title string for each index.
#  index 0 is preseason test days, mnaully added to list
   
    titles = []
    titles.append('NLS Test Days')
    
    count = 1
    for i in range(len(soup.find_all('tr')[1:9])):
        titles.append('NLS Race R0' + str(count))
        count += 1
#         titles.append(temp_tit)
    
    titles.append('NLS Alternative Date')

        
    return cln_yrs, cln_mons, cln_days, descriptions, titles

In [38]:
cln_yrs, cln_mons, cln_days, descriptions, titles = scrape_nls(soup)

In [39]:
cln_yrs

['2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023']

In [40]:
cln_mons

['3', '3', '4', '4', '6', '7', '9', '9', '10', '11']

In [41]:
cln_days

['12', '18', '1', '15', '17', '8', '10', '23', '7', '4']

In [42]:
descriptions

['Test- and Adjustment Drives',
 'NLS1: 68. ADAC Westfalenfahrt',
 'NLS2: 47. NIMEX DMV 4-Stunden-Rennen',
 'NLS3: 54. Adenauer ADAC Rundstrecken-Trophy',
 'NLS4: 46. RCM DMV Grenzlandrennen',
 'NLS5: ROWE 6h ADAC Ruhr-Pokal-Rennen',
 '12h Nürburgring\nRed Bull Formula Nürburgring\nNLS6: 63. ADAC ACAS Cup\nNLS7: 62. ADAC Reinoldus-Langstreckenrennen',
 'NLS8: 55. ADAC Barbarossapreis',
 'NLS9: 47. PAGID RACING DMV Münsterlandpokal',
 'Alternative Date']

In [43]:
titles

['NLS Test Days',
 'NLS Race R01',
 'NLS Race R02',
 'NLS Race R03',
 'NLS Race R04',
 'NLS Race R05',
 'NLS Race R06',
 'NLS Race R07',
 'NLS Race R08',
 'NLS Alternative Date']

In [44]:
circuit = 'Nürburgring Gesamtrecke' '\n' 'Nürburg, Rhineland-Palatinate, Germany'# 
print(circuit)

Nürburgring Gesamtrecke
Nürburg, Rhineland-Palatinate, Germany


In [45]:
## Creating and Populating the Calendar

In [46]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [47]:
def new_ical(series_name):
    
   
    series_name = Calendar()
    series_name.add('prodid', '-//My calendar product//example.com//')
    series_name.add('version', '2.0')
    
    return series_name



In [48]:
NLS = None
NLS = new_ical(NLS)
NLS

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')})

In [49]:


def make_events(cal):
    uid = 110
    circuit = 'Nürburgring Gesamtrecke' '\n' 'Nürburg, Rhineland-Palatinate, Germany'

    for title, description, year, month, day in zip(titles, descriptions, cln_yrs, cln_mons, cln_days):
        ievent = Event()
        ievent.add('summary', title) #Title of the event
        ievent.add('description', description)     #Description of event
        ievent.add('dtstart', datetime(int(year), int(month), int(day), 10, 0, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('dtend', datetime(int(year), int(month), int(day), 14, 0, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('dtstamp', datetime(2023, 3, 17, 0, 10, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('location', vText(circuit))
        ievent.add('uid', uid)
        uid += 1
        cal.add_component(ievent)
        
    return cal
    

In [50]:
NLS = make_events(NLS)
# NLS

In [51]:
def make_ics():
    directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
    print("ics file will be generated at ", directory)
    f = open(os.path.join(directory, 'NLS 2023.ics'), 'wb')
    f.write(NLS.to_ical())
    f.close()



In [52]:
make_ics()

ics file will be generated at  C:\Users\chris\Documents\Calendars/


In [53]:
# directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
# print("ics file will be generated at ", directory)
# f = open(os.path.join(directory, 'NLS 2023.ics'), 'wb')
# f.write(NLS.to_ical())
# f.close()
